In [0]:
# Initialization
import numpy as np
seed = 256
np.random.seed(seed)
import tensorflow as tf
tf.random.set_seed(seed)

In [2]:
# Imports
from sklearn.model_selection import KFold
import os
from keras.optimizers import Adam

Using TensorFlow backend.


In [0]:
# Google Drive mount
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# Links functions
%run /content/drive/My\ Drive/segmentation/functions.ipynb
%run /content/drive/My\ Drive/segmentation/models.ipynb
%run /content/drive/My\ Drive/segmentation/metrics.ipynb

In [0]:
# Config
batch_size = 1
create_data = False

dim_x = 96
dim_y = 96
dim_z = 128

dim = (dim_x, dim_y, dim_z)

data_path = '/content/drive/My Drive/segmentation/data'

In [0]:
# Create data
if create_data:
  create_train_masks(data_path, dim)
  create_train_imgs(data_path, dim)

In [0]:
# Load data
train_imgs, train_masks = load_train_data(data_path) 
train_imgs = train_imgs.astype(np.float32) / 255.
train_masks = vertebrae_to_spine(train_masks).astype(np.float32)

In [0]:
# Define folds
f = KFold(3, True, seed)

In [0]:
# Set up accuracies
pixel_acc = []
dice_acc = []
iou_acc = []
auc_acc = []

In [0]:
# K-fold cross validation
for train_index, test_index in f.split(train_imgs[:3]):
  print("TRAIN:", train_index, "TEST:", test_index)
  
  X_train, X_test = train_imgs[train_index], train_imgs[test_index]
  y_train, y_test = train_masks[train_index], train_masks[test_index]

  # Define model
  model = unet_3d((96, 96, 128, 1))

  # Compile model
  model.compile(optimizer=Adam(learning_rate=0.00001),loss=dice_coef_loss, metrics=[dice_coef])

  # Fit the model
  model.fit(X_train, y_train, epochs=1, batch_size=1, verbose=1)

  predictions = model.predict(X_test)

  score = pixel_wise_accuracy(y_test, predictions > 0.9)
  print(score)
  pixel_acc.append(score)

  score = dice(y_test, (predictions > 0.9).astype(np.float32))
  print(score)
  dice_acc.append(score)

  score = iou(y_test, (predictions > 0.9).astype(np.float32))
  print(score)
  iou_acc.append(score)

  score = auc(y_test, (predictions > 0.9).astype(np.float32))
  print(score)
  auc_acc.append(score)

print("Pixel-wise accuracy: %f(+/- %f)" % (np.mean(pixel_acc), np.std(pixel_acc)))
print("Dice coefficient: %f(+/- %f)" % (np.mean(dice_acc), np.std(dice_acc))) 
print("IOU: %f(+/- %f)" % (np.mean(iou_acc), np.std(iou_acc)))
print("AUC: %f(+/- %f)" % (np.mean(auc_acc), np.std(auc_acc)))       